In [12]:
import pandas as pd
from transformers import pipeline
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [13]:
data = pd.read_csv('/content/drive/MyDrive/IUT/IR/project/4. embedding & language models/train.csv')

sampled_data = data.sample(n=1000, random_state=42)
sentences = sampled_data['Title'].tolist()

In [14]:
from transformers import BertTokenizer, BertForMaskedLM
import torch

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForMaskedLM.from_pretrained('bert-base-uncased')

def generate_sentence(sentence, mask="[MASK]"):
    tokens = tokenizer.tokenize(sentence)

    masked_index = len(tokens) // 2
    tokens[masked_index] = mask

    input_ids = tokenizer.convert_tokens_to_ids(tokens)

    input_ids_tensor = torch.tensor([input_ids])

    with torch.no_grad():
        outputs = model(input_ids_tensor)
        predictions = outputs.logits

    predicted_index = torch.argmax(predictions[0, masked_index]).item()
    predicted_token = tokenizer.convert_ids_to_tokens([predicted_index])[0]

    tokens[masked_index] = predicted_token

    return tokenizer.convert_tokens_to_string(tokens)

augmented_sentences = []
for sentence in sentences:
    new_sentence = generate_sentence(sentence)
    augmented_sentences.append(new_sentence)
    augmented_sentences.append(new_sentence)


augmented_data = pd.DataFrame({
    'Original': sentences * 2,
    'Generated': augmented_sentences
})

augmented_data.to_csv('/content/drive/MyDrive/IUT/IR/project/4. embedding & language models/augmented_dataset.csv', index=False)

print("Dataset has been saved as 'augmented_dataset.csv'")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Dataset has been saved as 'augmented_dataset.csv'
